## **Extract features from image usikng ResNet**

### **Import necessary libraries**

In [28]:
import os 
import sys

import pandas as pd 
import numpy as np 

import matplotlib.pyplot as plt 
%matplotlib inline 
import seaborn as sns 

import warnings 
warnings.filterwarnings('ignore')

from PIL import Image 

import torch
import torchvision 
import torchvision.transforms as transforms 
import torchvision.models as models 
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn 
from torch.autograd import Variable


In [29]:
current_cwd = os.getcwd()
lst = current_cwd.split('\\', 2)
new_cwd = '/'.join(lst[:2])
os.chdir(new_cwd)
sys.path.append(new_cwd + '/code')

In [30]:
IMAGES_PATH = 'data/flickr8k/Images'
CAPTIONS_PATH = 'data/flickr8k/captions.txt'

data = pd.read_csv(CAPTIONS_PATH)
data

,image,caption
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...
...,...,...
40450,997722733_0cb5439472.jpg,A man in a pink shirt climbs a rock face
40451,997722733_0cb5439472.jpg,A man is rock climbing high in the air .
40452,997722733_0cb5439472.jpg,A person in a red shirt climbing up a rock fac...
40453,997722733_0cb5439472.jpg,A rock climber in a red shirt .


In [31]:
images_data = data.drop_duplicates(subset=['image'])
images_data

,image,caption
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...
5,1001773457_577c3a7d70.jpg,A black dog and a spotted dog are fighting
10,1002674143_1b742ab4b8.jpg,A little girl covered in paint sits in front o...
15,1003163366_44323f5815.jpg,A man lays on a bench while his dog sits by him .
20,1007129816_e794419615.jpg,A man in an orange hat starring at something .
...,...,...
40430,990890291_afc72be141.jpg,A man does a wheelie on his bicycle on the sid...
40435,99171998_7cc800ceef.jpg,A group is sitting around a snowy crevasse .
40440,99679241_adc853a5c0.jpg,A grey bird stands majestically on a beach whi...
40445,997338199_7343367d7f.jpg,A person stands near golden walls .


### **Load the ResNet18**

In [32]:
# Load the pretrained model
resnet18 = models.resnet18(pretrained=True)

for i, x in resnet18.named_children():
    print(f'NAME: \n {i}')
    print(f'CONTENT: \n {x}')
    print('-'*100)

NAME: 
 conv1
CONTENT: 
 Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
----------------------------------------------------------------------------------------------------
NAME: 
 bn1
CONTENT: 
 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
----------------------------------------------------------------------------------------------------
NAME: 
 relu
CONTENT: 
 ReLU(inplace=True)
----------------------------------------------------------------------------------------------------
NAME: 
 maxpool
CONTENT: 
 MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
----------------------------------------------------------------------------------------------------
NAME: 
 layer1
CONTENT: 
 Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (re

In [33]:
# Use the model object to select the desired layer
layer = resnet18._modules.get('avgpool')

In [34]:
resize = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])   # mean and std of ImageNet
to_tensor = transforms.ToTensor()

In [35]:
def get_vector_resnet18(img_name):
    # Load the image with Pillow library 
    img = Image.open(img_name).convert('RGB')
    
    # Create a PyTorch Variable with the transformed image 
    transformed_img = Variable(normalize(to_tensor(resize(img))).unsqueeze(0))
    
    # Create a vector of zeros that will hold our feature vector 
    # The 'avgpool' layer has an output size of 512
    my_embedding = torch.zeros(512)
    
    # Define a function that will copy the output of a layer 
    def copy_data(m, i, o):
        my_embedding.copy_(o.data.view(-1))
        
    # Attach that function to our selected layer 
    h = layer.register_forward_hook(copy_data)
    
    # Run the model on our transformed image 
    resnet18(transformed_img)
    
    # Detach our copy function from the layer 
    h.remove()
    
    # Return the feature vector 
    return my_embedding

In [36]:
resnet18_image = {}
cnt = 0
for img in images_data.image:
    resnet18_image[img] = get_vector_resnet18(IMAGES_PATH + '/' + img)
    cnt += 1
    if cnt % 1000 == 0:
        print(f"Completed {cnt} images")

Completed 1000 images
Completed 2000 images
Completed 3000 images
Completed 4000 images
Completed 5000 images
Completed 6000 images
Completed 7000 images
Completed 8000 images


In [37]:
resnet18_features = images_data

In [38]:
resnet18_features['image features'] = resnet18_features.image.map(resnet18_image)

In [39]:
resnet18_features = resnet18_features[['image', 'image features']].reindex()
with open('Image features/resnet18.csv', mode='w', encoding='utf-8') as file: 
    resnet18_features.to_csv(file, index=False)

### **Load the ResNet34**

In [40]:
# Load the pretrained model
resnet34 = models.resnet34(pretrained=True)

for i, x in resnet34.named_children():
    print(f'NAME: \n {i}')
    print(f'CONTENT: \n {x}')
    print('-'*100)

NAME: 
 conv1
CONTENT: 
 Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
----------------------------------------------------------------------------------------------------
NAME: 
 bn1
CONTENT: 
 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
----------------------------------------------------------------------------------------------------
NAME: 
 relu
CONTENT: 
 ReLU(inplace=True)
----------------------------------------------------------------------------------------------------
NAME: 
 maxpool
CONTENT: 
 MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
----------------------------------------------------------------------------------------------------
NAME: 
 layer1
CONTENT: 
 Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (re

In [41]:
# Use the model object to select the desired layer
layer = resnet34._modules.get('avgpool')

In [42]:
resize = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])   # mean and std of ImageNet
to_tensor = transforms.ToTensor()

In [43]:
def get_vector_resnet34(img_name):
    # Load the image with Pillow library 
    img = Image.open(img_name).convert('RGB')
    
    # Create a PyTorch Variable with the transformed image 
    transformed_img = Variable(normalize(to_tensor(resize(img))).unsqueeze(0))
    
    # Create a vector of zeros that will hold our feature vector 
    # The 'avgpool' layer has an output size of 512
    my_embedding = torch.zeros(512)
    
    # Define a function that will copy the output of a layer 
    def copy_data(m, i, o):
        my_embedding.copy_(o.data.view(-1))
        
    # Attach that function to our selected layer 
    h = layer.register_forward_hook(copy_data)
    
    # Run the model on our transformed image 
    resnet34(transformed_img)
    
    # Detach our copy function from the layer 
    h.remove()
    
    # Return the feature vector 
    return my_embedding

In [44]:
resnet34_image = {}
cnt = 0
for img in images_data.image:
    resnet34_image[img] = get_vector_resnet34(IMAGES_PATH + '/' + img)
    cnt += 1
    if cnt % 1000 == 0:
        print(f"Completed {cnt} images")

Completed 1000 images
Completed 2000 images
Completed 3000 images
Completed 4000 images
Completed 5000 images
Completed 6000 images
Completed 7000 images
Completed 8000 images


In [45]:
resnet34_features = images_data

In [46]:
resnet34_features['image features'] = resnet34_features.image.map(resnet34_image)

In [47]:
resnet34_features = resnet34_features[['image', 'image features']].reindex()
with open('Image features/resnet34.csv', mode='w', encoding='utf-8') as file: 
    resnet34_features.to_csv(file, index=False)

### **Load the ResNet50**

In [49]:
# Load the pretrained model
resnet50 = models.resnet50(pretrained=True)

for i, x in resnet50.named_children():
    print(f'NAME: \n {i}')
    print(f'CONTENT: \n {x}')
    print('-'*100)

NAME: 
 conv1
CONTENT: 
 Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
----------------------------------------------------------------------------------------------------
NAME: 
 bn1
CONTENT: 
 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
----------------------------------------------------------------------------------------------------
NAME: 
 relu
CONTENT: 
 ReLU(inplace=True)
----------------------------------------------------------------------------------------------------
NAME: 
 maxpool
CONTENT: 
 MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
----------------------------------------------------------------------------------------------------
NAME: 
 layer1
CONTENT: 
 Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64,

In [50]:
# Use the model object to select the desired layer
layer = resnet50._modules.get('avgpool')

In [51]:
resize = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])   # mean and std of ImageNet
to_tensor = transforms.ToTensor()

In [52]:
def get_vector_resnet50(img_name):
    # Load the image with Pillow library 
    img = Image.open(img_name).convert('RGB')
    
    # Create a PyTorch Variable with the transformed image 
    transformed_img = Variable(normalize(to_tensor(resize(img))).unsqueeze(0))
    
    # Create a vector of zeros that will hold our feature vector 
    # The 'avgpool' layer has an output size of 2048
    my_embedding = torch.zeros(2048)
    
    # Define a function that will copy the output of a layer 
    def copy_data(m, i, o):
        my_embedding.copy_(o.data.view(-1))
        
    # Attach that function to our selected layer 
    h = layer.register_forward_hook(copy_data)
    
    # Run the model on our transformed image 
    resnet50(transformed_img)
    
    # Detach our copy function from the layer 
    h.remove()
    
    # Return the feature vector 
    return my_embedding

In [53]:
resnet50_image = {}
cnt = 0
for img in images_data.image:
    resnet50_image[img] = get_vector_resnet50(IMAGES_PATH + '/' + img)
    cnt += 1
    if cnt % 1000 == 0:
        print(f"Completed {cnt} images")

Completed 1000 images
Completed 2000 images
Completed 3000 images
Completed 4000 images
Completed 5000 images
Completed 6000 images
Completed 7000 images
Completed 8000 images


In [54]:
resnet50_features = images_data

In [55]:
resnet50_features['image features'] = resnet50_features.image.map(resnet50_image)

In [56]:
resnet50_features = resnet50_features[['image', 'image features']].reindex()
with open('Image features/resnet50.csv', mode='w', encoding='utf-8') as file: 
    resnet50_features.to_csv(file, index=False)

### **Load the ResNet101**

In [57]:
# Load the pretrained model
resnet101 = models.resnet101(pretrained=True)

for i, x in resnet101.named_children():
    print(f'NAME: \n {i}')
    print(f'CONTENT: \n {x}')
    print('-'*100)

Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to C:\Users\ASUS/.cache\torch\hub\checkpoints\resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:24<00:00, 7.18MB/s] 


NAME: 
 conv1
CONTENT: 
 Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
----------------------------------------------------------------------------------------------------
NAME: 
 bn1
CONTENT: 
 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
----------------------------------------------------------------------------------------------------
NAME: 
 relu
CONTENT: 
 ReLU(inplace=True)
----------------------------------------------------------------------------------------------------
NAME: 
 maxpool
CONTENT: 
 MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
----------------------------------------------------------------------------------------------------
NAME: 
 layer1
CONTENT: 
 Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64,

In [58]:
# Use the model object to select the desired layer
layer = resnet101._modules.get('avgpool')

In [59]:
resize = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])   # mean and std of ImageNet
to_tensor = transforms.ToTensor()

In [60]:
def get_vector_resnet101(img_name):
    # Load the image with Pillow library 
    img = Image.open(img_name).convert('RGB')
    
    # Create a PyTorch Variable with the transformed image 
    transformed_img = Variable(normalize(to_tensor(resize(img))).unsqueeze(0))
    
    # Create a vector of zeros that will hold our feature vector 
    # The 'avgpool' layer has an output size of 2048
    my_embedding = torch.zeros(2048)
    
    # Define a function that will copy the output of a layer 
    def copy_data(m, i, o):
        my_embedding.copy_(o.data.view(-1))
        
    # Attach that function to our selected layer 
    h = layer.register_forward_hook(copy_data)
    
    # Run the model on our transformed image 
    resnet101(transformed_img)
    
    # Detach our copy function from the layer 
    h.remove()
    
    # Return the feature vector 
    return my_embedding

In [61]:
resnet101_image = {}
cnt = 0
for img in images_data.image:
    resnet101_image[img] = get_vector_resnet101(IMAGES_PATH + '/' + img)
    cnt += 1
    if cnt % 1000 == 0:
        print(f"Completed {cnt} images")

Completed 1000 images
Completed 2000 images
Completed 3000 images
Completed 4000 images
Completed 5000 images
Completed 6000 images
Completed 7000 images
Completed 8000 images


In [62]:
resnet101_features = images_data

In [63]:
resnet101_features['image features'] = resnet101_features.image.map(resnet101_image)

In [64]:
resnet101_features = resnet101_features[['image', 'image features']].reindex()
with open('Image features/resnet101.csv', mode='w', encoding='utf-8') as file: 
    resnet101_features.to_csv(file, index=False)

### **Load the ResNet152**

In [65]:
resnet152 = models.resnet152(pretrained=True)

for i, x in resnet152.named_children():
    print(f'NAME: \n {i}')
    print(f'CONTENT: \n {x}')
    print('-'*100)

Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to C:\Users\ASUS/.cache\torch\hub\checkpoints\resnet152-394f9c45.pth
100%|██████████| 230M/230M [00:44<00:00, 5.37MB/s] 


NAME: 
 conv1
CONTENT: 
 Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
----------------------------------------------------------------------------------------------------
NAME: 
 bn1
CONTENT: 
 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
----------------------------------------------------------------------------------------------------
NAME: 
 relu
CONTENT: 
 ReLU(inplace=True)
----------------------------------------------------------------------------------------------------
NAME: 
 maxpool
CONTENT: 
 MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
----------------------------------------------------------------------------------------------------
NAME: 
 layer1
CONTENT: 
 Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64,

In [66]:
layer = resnet152._modules.get('avgpool')

In [67]:
resize = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])   # mean and std of ImageNet
to_tensor = transforms.ToTensor()

In [68]:
def get_vector_resnet152(img_name):
    # Load the image with Pillow library 
    img = Image.open(img_name).convert('RGB')
    
    # Create a PyTorch Variable with the transformed image 
    transformed_img = Variable(normalize(to_tensor(resize(img))).unsqueeze(0))
    
    # Create a vector of zeros that will hold our feature vector 
    # The 'avgpool' layer has an output size of 2048
    my_embedding = torch.zeros(2048)
    
    # Define a function that will copy the output of a layer 
    def copy_data(m, i, o):
        my_embedding.copy_(o.data.view(-1))
        
    # Attach that function to our selected layer 
    h = layer.register_forward_hook(copy_data)
    
    # Run the model on our transformed image 
    resnet152(transformed_img)
    
    # Detach our copy function from the layer 
    h.remove()
    
    # Return the feature vector 
    return my_embedding

In [69]:
resnet152_image = {}
cnt = 0
for img in images_data.image:
    resnet152_image[img] = get_vector_resnet152(IMAGES_PATH + '/' + img)
    cnt += 1
    if cnt % 1000 == 0:
        print(f"Completed {cnt} images")

Completed 1000 images
Completed 2000 images
Completed 3000 images
Completed 4000 images
Completed 5000 images
Completed 6000 images
Completed 7000 images
Completed 8000 images


In [70]:
resnet152_features = images_data

In [71]:
resnet152_features['image features'] = resnet152_features.image.map(resnet152_image)

In [72]:
resnet152_features = resnet152_features[['image', 'image features']].reindex()
with open('Image features/resnet152.csv', mode='w', encoding='utf-8') as file: 
    resnet152_features.to_csv(file, index=False)